<a href="https://colab.research.google.com/github/chernoskutov1989dv/Self-Education/blob/main/words_prediction_se_21_dmitry_chernoskutov.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np

from tensorflow.keras.layers import Dense, SimpleRNN, Input
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import Tokenizer, text_to_word_sequence
from tensorflow.keras.utils import to_categorical


In [ ]:
open ("/content/train_data_true.txt")

<_io.TextIOWrapper name='/content/train_data_true.txt' mode='r' encoding='UTF-8'>

In [ ]:
open ("/content/train_data_false.txt")

<_io.TextIOWrapper name='/content/train_data_false.txt' mode='r' encoding='UTF-8'>

In [ ]:
with open('train_data_true.txt', 'r', encoding='utf-8') as f:
    texts = f.read()
    texts = texts.replace('\ufeff', '')  # убираем первый невидимый символ


In [ ]:
#with open('train_data_false.txt', 'r', encoding='utf-8') as f:
    #texts = f.read()
    #texts = texts.replace('\ufeff', '') 

In [ ]:
maxWordsCount = 1000
tokenizer = Tokenizer(num_words=maxWordsCount, filters='!–"—#$%&amp;()*+,-./:;<=>?@[\\]^_`{|}~\t\n\r«»',
                      lower=True, split=' ', char_level=False)
tokenizer.fit_on_texts([texts])

dist = list(tokenizer.word_counts.items())
print(dist[:10])


[('думайте', 1), ('позитивно', 4), ('и', 29), ('верьте', 3), ('в', 23), ('свою', 3), ('способность', 1), ('достигать', 1), ('отличных', 1), ('результатов', 1)]


In [ ]:
data = tokenizer.texts_to_sequences([texts])
res = to_categorical(data[0], num_classes=maxWordsCount)
print(res.shape)


(989, 1000)


In [ ]:
inp_words = 3
n = res.shape[0] - inp_words

X = np.array([res[i:i + inp_words, :] for i in range(n)])
Y = res[inp_words:]

In [ ]:
model = Sequential()
model.add(Input((inp_words, maxWordsCount)))
model.add(SimpleRNN(128, activation='tanh'))
model.add(Dense(maxWordsCount, activation='softmax'))
model.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 simple_rnn_5 (SimpleRNN)    (None, 128)               144512    
                                                                 
 dense_5 (Dense)             (None, 1000)              129000    
                                                                 
Total params: 273,512
Trainable params: 273,512
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer='adam')

In [ ]:
history = model.fit(X, Y, batch_size=32, epochs=50)

Epoch 1/50
31/31 [==============================] - 1s 9ms/step - loss: 6.8911 - accuracy: 0.0132
Epoch 2/50
31/31 [==============================] - 0s 8ms/step - loss: 6.3346 - accuracy: 0.0487
Epoch 3/50
31/31 [==============================] - 0s 10ms/step - loss: 5.8501 - accuracy: 0.0507
Epoch 4/50
31/31 [==============================] - 0s 9ms/step - loss: 5.6752 - accuracy: 0.0396
Epoch 5/50
31/31 [==============================] - 0s 9ms/step - loss: 5.5217 - accuracy: 0.0477
Epoch 6/50
31/31 [==============================] - 0s 10ms/step - loss: 5.3516 - accuracy: 0.0730
Epoch 7/50
31/31 [==============================] - 0s 9ms/step - loss: 5.1445 - accuracy: 0.1136
Epoch 8/50
31/31 [==============================] - 0s 9ms/step - loss: 4.8774 - accuracy: 0.1349
Epoch 9/50
31/31 [==============================] - 0s 10ms/step - loss: 4.5333 - accuracy: 0.1968
Epoch 10/50
31/31 [==============================] - 0s 9ms/step - loss: 4.1257 - accuracy: 0.2718
Epoch 11/50
31/3

In [ ]:
def buildPhrase(texts, str_len=50):
    res = texts
    data = tokenizer.texts_to_sequences([texts])[0]
    for i in range(str_len):
        x = to_categorical(data[i: i + inp_words], num_classes=maxWordsCount)  # преобразуем в One-Hot-encoding
        inp = x.reshape(1, inp_words, maxWordsCount)

        pred = model.predict(inp)
        indx = pred.argmax(axis=1)[0]
        data.append(indx)

        res += " " + tokenizer.index_word[indx]  # дописываем строку

    return res
res = buildPhrase("Хорошее настроение добавляет")
print(res)

1/1 [==============================] - 0s 18ms/step
Хорошее настроение добавляет годы к вашей жизни весну к вашему шагу искорку в ваших глазах природные способности важны но вы можете многого достичь и без них если у вас есть два варианта быть положительным или отрицательным оптимистом или пессимистом практически нет ничего невозможного в этом мире если вы просто посмотрите на жизнь позитивно
